# User Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the cosine correlation between the two users
* The algorithm is nearly identical to the algorithm in `ItemCF`, with the following exceptions
  * we transpose users and items
  * the similarity matrix is too big to store in memory, so we generate weights on the fly


In [1]:
name = "UserCF";

In [2]:
using DataStructures
using LRUCache
using LSHFunctions
using Random

In [3]:
using NBInclude
@nbinclude("ItemCollaborativeFilteringBase.ipynb");

## Compute nearest neighbors using Locality Sensitive Hashing

In [4]:
# Use locality sensitive hashing to reduce the computational cost of finding nearest neighbors
# TODO automatically adjust scale based on the number of items
# TODO save hash function or random seed
const n_hashes = 128
const hashfn = L2Hash(n_hashes, scale = 0.01)

@memoize function get_hash_buckets(R)
    hash_buckets = [DefaultDict(() -> Int64[]) for i = 1:n_hashes]
    @showprogress for j = 1:size(R)[2]
        hashes = hashfn(R[:, j])
        for i = 1:n_hashes
            push!(hash_buckets[i][hashes[i]], j)
        end
    end
    hash_buckets
end

function get_hash_neighbors(R, j)
    hash_buckets = get_hash_buckets(R)
    hash = hashfn(R[:, j])
    vcat([hash_buckets[i][hash[i]] for i = 1:length(hash)]...)
end;

In [5]:
@memoize function get_norms(R)
    norms = map(norm, eachslice(R, dims = 2))
    norms[norms.==0] .= 1 # prevent division by 0
    norms
end;

# @memoize LRU{Tuple{Any,Any},Any}(maxsize = 1) function get_abs_neighborhood_cache(R, K)
#     # we need to preallocate the dict to avoid resizing
#     # while accessing from multiple threads
#     d = Dict{Int64,Any}(i => nothing for i = 1:maximum(size(R)[2]))
# end

# function get_abs_neighborhood(item, R, K)
#     cache = get_abs_neighborhood_cache(R, K)
#     val = cache[item]
#     if val == nothing
#         norms = get_norms(R)
#         weights = zeros(size(R)[2])
#         mask = get_hash_neighbors(R, item)
#         weights[mask] = vec(R[:, item]' * R[:, mask])
#         weights = weights ./ norms ./ norms[item]
#         weights[item] = Inf
#         order = partialsortperm(abs.(weights), 2:K+1, rev = true)
#         cache[item] = (order, weights[order])
#         return (order, weights[order])
#     end
#     val
# end;

@memoize function get_abs_neighborhood(item, R, K)
    norms = get_norms(R)
    weights = zeros(size(R)[2])
    mask = get_hash_neighbors(R, item)
    weights[mask] = vec(R[:, item]' * R[:, mask])
    weights = weights ./ norms ./ norms[item]
    weights[item] = Inf
    order = partialsortperm(abs.(weights), 2:K+1, rev = true)
    (order, weights[order])
end;

## Transform the dataset

In [6]:
function Base.adjoint(x::RatingsDataset)
    RatingsDataset(x.item, x.user, x.rating)
end;

In [7]:
function filter_items(x::RatingsDataset, sparsity)
    # return a randomly chosen subset of x with the given sparsity
    # the subset will minimize the number of distinct items
    Random.seed!(20220102 * hash(sparsity))
    items = shuffle(sort(collect(Set(x.item))))
    n = Int(round(length(items) * sparsity))
    valid_items = Set(items[1:n])
    mask = [x.item[i] ∈ valid_items for i = 1:length(x.item)]
    RatingsDataset(x.user[mask], x.item[mask], x.rating[mask])
end;

In [8]:
function filter_items(dest::RatingsDataset, source::RatingsDataset)
    # return the subset of dest whose items are im source
    items = collect(Set(source.item))
    valid_items = Set(items)
    mask = [dest.item[i] ∈ valid_items for i = 1:length(dest.item)]
    RatingsDataset(dest.user[mask], dest.item[mask], dest.rating[mask])
end;

## Training algorithm

In [9]:
function optimize_model(param)
    # unpack parameters
    training = get_training(param.training_residuals)'
    validation = get_validation(param.validation_residuals)'
    # running on the entire dataset is too computationally expensive, so we subset it
    # training = filter_items(training, 0.1)
    validation = filter_items(filter_items(validation, training), 0.001)
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )
    K = param.K
    neighborhood_types = Dict("abs" => get_abs_neighborhood)
    neighborhoods = i -> neighborhood_types[param.neighborhood_type](i, R, K)

    # generate the hash bucket cache 
    @debug "generating hash buckets"
    get_hash_buckets(R)
    hash_neighborhood_sizes = zeros(size(R)[2])
    @tprogress Threads.@threads for j = 1:size(R)[2]
        hash_neighborhood_sizes[j] = length(get_hash_neighbors(R, j))
    end
    @debug "average hash neighborhood size $(mean(hash_neighborhood_sizes))"

    # optimize hyperparameters
    function validation_mse(λ)
        pred = collaborative_filtering(training, validation, neighborhoods, λ)
        truth = validation.rating
        β = pred \ truth
        loss = mse(truth, pred .* β)
        @debug "loss: $loss β: $β: λ $λ"
        loss
    end
    res = optimize(
        validation_mse,
        param.λ,
        LBFGS(),
        autodiff = :forward,
        Optim.Options(show_trace = true, extended_trace = true),
    )
    param.λ = Optim.minimizer(res)

    # save predictions
    training = get_training(param.training_residuals)'
    inference = get_inference()'
    preds = collaborative_filtering(training, inference, neighborhoods, param.λ)
    sparse_preds = sparse(inference.user, inference.item, preds)'
    function model(users, items, predictions)
        result = zeros(length(users))
        for i = 1:length(users)
            if users[i] <= size(predictions)[1] && items[i] <= size(predictions)[2]
                result[i] = predictions[users[i], items[i]]
            end
        end
        result
    end
    write_predictions(
        (users, items) -> model(users, items, sparse_preds),
        outdir = param.name,
        residual_alphas = param.validation_residuals,
        save_training = true,
    )
    write_params(to_dict(param), outdir = param.name)
end;

## Setup hyperparameters

In [10]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
user_cf_params = [[
        cf_params(
            name = "UserCF.$K",
            training_residuals = ["UserItemBiases"],
            validation_residuals = ["UserItemBiases"],
            neighborhood_type = "abs",
            S = "",
            K = K,
            λ = [1.0426196637532286, 0.9279875295690789, 1.2651483332610922],
        ) for K in downcast_to_int.([2^10])
    ];
];

In [11]:
# 2^14 -> 1.477202

## Train models

In [ ]:
for param in user_cf_params
    optimize_model(param)
end

[ Debug: 20220102 18:16:33 generating hash buckets
Progress: 100%|█████████████████████████████████████████| Time: 0:00:40
Progress: 100%|███████████████████████████| Time: 0:00:27 ( 0.24 ms/it)
[ Debug: 20220102 18:17:43 average hash neighborhood size 16034.467805478405
Progress: 100%|███████████████████████████| Time: 0:00:34 ( 0.33  s/it)
[ Debug: 20220102 18:18:28 loss: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.4939208019927832,0.007607559709047942,-0.004345672032194704,0.0012035487988274426) β: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.8931566308979513,0.9431462094489499,-1.0124291960099738,0.31404905918623205): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, I

Iter     Function value   Gradient norm 
     0     1.493921e+00     7.607560e-03
 * Current step size: 1.0
 * time: 0.020226001739501953
 * g(x): [0.007607559709047942, -0.004345672032194704, 0.0012035487988274426]
 * x: [1.0426196637532286, 0.9279875295690789, 1.2651483332610922]


Progress: 100%|███████████████████████████| Time: 0:00:03 (35.04 ms/it)
[ Debug: 20220102 18:18:40 loss: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.493845942230105,0.006885733887403228,-0.004119089580470284,0.0010156116577587485) β: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.8812395038895586,0.9349676379516229,-1.011190294798933,0.3112417792559824): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.0350121040441806,1.0,0.0,0.0), Dual{ForwardDif

     1     1.493471e+00     1.612082e-03
 * Current step size: 11.29779375580281
 * time: 45.94201993942261
 * g(x): [-0.000655185196638668, -0.001612081905140529, -0.000948173893788669]
 * x: [0.9566710231754497, 0.9770840359191751, 1.2515508871568957]


Progress: 100%|███████████████████████████| Time: 0:00:02 (28.15 ms/it)
[ Debug: 20220102 18:19:24 loss: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.4934375018294124,-0.0003005680564744634,-0.0006868571603468091,-0.0008520769581940923) β: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.747472925081674,0.853754421699017,-1.009567502921913,0.2811731117892241): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(0.95960001843837,1.0,0.0,0.0), Dual{ForwardDi

...

Progress: 100%|███████████████████████████| Time: 0:00:03 (30.42 ms/it)
[ Debug: 20220102 18:24:53 loss: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.4927553028211116,-5.583376001367452e-7,-8.749505648960332e-7,-5.520576360643534e-8) β: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.843923522494299,1.0354590177805811,-1.0835314156470515,0.07813476126265183): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(0.5080489244958671,1.0,0.0,0.0), Dual{Forward

    12     1.492755e+00     7.699239e-08
 * Current step size: 1.199133280634837
 * time: 406.99757981300354
 * g(x): [-7.699238575623191e-8, 6.611856764030993e-8, -5.363951976938329e-9]
 * x: [0.5079976461796449, 1.0200242194772209, 5.263560355222637]


Progress: 100%|███████████████████████████| Time: 0:00:03 (30.47 ms/it)
[ Debug: 20220102 18:25:25 loss: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.4927553027994127,-6.544069563114265e-10,7.855685404857281e-10,-5.773590779451567e-11) β: Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(1.8439241898774485,1.0354821827746596,-1.083552072066929,0.07812148648228062): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#40"{var"#36#38"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, SparseMatrixCSC{Float64, Int64}}}, Float64}}(0.5079964176704305,1.0,0.0,0.0), Dual{Forwa

    13     1.492755e+00     3.285038e-10
 * Current step size: 1.0128758767579853
 * time: 439.1663308143616
 * g(x): [3.2850377707735825e-10, -5.564982761981516e-11, 1.0585363383248056e-11]
 * x: [0.5079964018522972, 1.0200229011825892, 5.263592048627993]


Progress:   0%|▏                          |  ETA: 22:21:27 ( 0.71  s/it)t)m